# 13.3 BLEU Score

앞서 언어 모델(Language Model)의 성능 측정을 위한 평가 방법(evaluation metric)으로 펄플렉서티(perplexity, PPL)를 소개한 바 있습니다. 기계 번역기에도 PPL을 평가에 사용할 수는 있지만, PPL은 번역의 성능을 직접적으로 반영하는 수치라 보기엔 어렵습니다.

자연어 처리에서는 그 외에도 수많은 평가 방법들이 존재하는데, 이번 챕터에서는 기계 번역의 성능이 얼마나 뛰어난가를 측정하기 위해 사용되는 대표적인 방법인 BLEU(Bilingual Evaluation Understudy) 대해서 배워보겠습니다. 이하 BLEU라고 명명합니다. 이번 챕터에서는 BLEU의 개념에 대해서 이해하고, 가장 기본적인 BLEU를 구현해보도록 하겠습니다.

이번 챕터는 논문 BLEU: a Method for Automatic Evaluation of Machine Translation를 참고로 하여 작성되었습니다

In [1]:
from collections import Counter
import numpy as np
from nltk import ngrams

### 1. BLEU(Bilingual Evaluation Understudy)
BLEU는 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하여 번역에 대한 성능을 측정하는 방법입니다. 측정 기준은 n-gram에 기반합니다. n-gram의 정의는 n-gram 언어 모델 챕터를 참고하시기 바랍니다.

BLEU는 완벽한 방법이라고는 할 수는 없지만 몇 가지 이점을 가집니다. 
- 언어에 구애받지 않고 사용할 수 있으며, 
- 계산 속도가 빠릅니다. 
- BLEU는 PPL과는 달리 높을 수록 성능이 더 좋음을 의미합니다. 

BLEU를 이해하기 위해 기계 번역 성능 평가를 위한 몇 가지 직관적인 방법을 먼저 제시하고, 문제점을 보완해나가는 방식으로 설명합니다.

##### 1) 단어 개수 카운트로 측정하기(Unigram Precision)
한국어-영어 번역기의 성능을 측정한다고 가정해봅시다. 두 개의 기계 번역기가 존재하고 두 기계 번역기에 같은 한국어 문장을 입력하여 번역된 영어 문장의 성능을 측정하고자 합니다. 번역된 문장을 각각 Candidate1, 2라고 해봅시다. 이 문장의 성능을 평가하기 위해서는 정답으로 비교되는 문장이 있어야 합니다. 세 명의 사람에게 한국어를 보고 영작해보라고 하여 세 개의 번역 문장을 만들어냈습니다. 이 세 문장을 각각 Reference1, 2, 3라고 해봅시다.

편의상 Candidate를 Ca로, Reference를 Ref로 축약하여 부르겠습니다. 이제 Ca 1, 2를 Ref 1, 2, 3과 비교하여 성능을 측정하고자 합니다. 가장 직관적인 성능 평가 방법은 Ref 1, 2, 3 중 어느 한 문장이라도 등장한 단어의 개수를 Ca에서 세는 것입니다. 그리고 그 후에 Ca의 모든 단어의 카운트의 합. 즉, Ca에서의 총 단어의 수으로 나눠줍니다.

이러한 측정 방법을 유니그램 정밀도(Unigram Precision)라고 합니다.

##### 2) 중복을 제거하여 보정하기(Modified Unigram Precision)

##### 3) 보정된 유니그램 정밀도 (Modified Unigram Precision) 구현하기

In [4]:
# 단순 카운터 함수
def simple_count(tkns, n): # 토큰화 된 candidate 문장, n-gram에서의 n 이 두 가지를 인자로 받음.
    return Counter(ngrams(tkns,n))  #문장에서 n-gram을 카운트

위 함수는 토큰화 된 문장을 입력받아서 문장 내의 n-gram의 개수를 카운트하는 함수입니다. 구하고자 하는 것은 유니그램 정밀도이므로 카운트하고자 하는 n-gram의 단위를 결정하는 simple_count 함수의 두번째 인자인 n의 값을 1로 하여 함수를 실행하면 됩니다. Example 1의 Ca1를 가져와 함수가 어떤 결과를 출력하는지 확인해봅시다

In [5]:
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."
tkns = candidate.split() #단어 토큰화
result = simple_count(tkns,1) #토큰화 된 문장, 유니그램의 개수를 구하고자 한다면 n=1
print(result)

Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


위의 출력 결과는 모든 유니그램을 카운트한 결과를 보여줍니다. 대부분의 유니그램이 1개씩 카운트되었으나 유니그램 the는 문장에서 3번 등장하였으므로 유일하게 3의 값을 가집니다. 이번에는 Example 2의 Ca를 가지고 함수를 수행해봅시다.

In [6]:
candidate ='the the the the the the the'
tkns = candidate.split()
result = simple_count(tkns,1)
print(result)

Counter({('the',): 7})


simple_count 함수는 단순 카운트를 수행하므로 the에 대해서 7이라는 카운트 값을 리턴합니다. Count에 대한 함수를 구현하였으니 이번에는 Countclip을 아래의 count_clip 이름을 가진 함수로 구현해보겠습니다.

In [16]:
def count_clip(candidate, reference_list, n):
    cnt_ca = simple_count(candidate, n)
    # Ca 문장에서 n-gram count
    temp = dict()
    
    for ref in reference_list: # 다수의 Ref 문장에 대해서 이하 반복
        cnt_ref = simple_count(ref, n)
        # Ref 문장에서 n-gram 카운트
        
        for n_gram in cnt_ref: # 모든 Ref에 대해서 비교하여 특정 n-gram이 하나의 Ref에 가장 많이 등장한 횟수를 저장
            if n_gram in temp:
                temp[n_gram] = max(cnt_ref[n_gram], temp[n_gram]) # max_ref_count
            else:
                temp[n_gram] = cnt_ref[n_gram]
    return {
        n_gram: min(cnt_ca.get(n_gram,0), temp.get(n_gram,0)) for n_gram in cnt_ca
    # count_clip=min(count, max_ref_count)
        # 위의 get은 찾고자 하는 n-gram이 없으면 0을 반환한다.
    }

count_clip 함수는 candidate 문장과 reference 문장들, 그리고 카운트 단위가 되는 n-gram에서의 n의 값 이 세 가지를 인자로 입력받아서 countclip을 수행합니다. 여기서는 유니그램 정밀도를 구현하고 있으므로 역시나 n=1로 하여 함수를 실행하면 됩니다.

또한 count_clip 함수 내부에는 기존에 구현했던 simple_count 함수가 사용된 것을 확인할 수 있습니다. Countclip을 구하기 위해서는 Max_Ref_Count값과 비교하기 위해 Count값이 필요하기 때문입니다. Example 2를 통해 함수가 정상 작동되는지 확인해봅시다.

In [17]:
candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]

result = count_clip(candidate.split(), list(map(lambda ref: ref.split(), references)),1)
print(result)

{('the',): 2}


동일한 예제 문장에 대해서 위의 simple_count 함수는 the가 7개로 카운트되었던 것과는 달리 이번에는 2개로 카운트되었습니다. 이제 위의 두 함수를 사용하여 예제 문장에 대해서 보정된 정밀도를 연산하는 함수를 modified_precision란 이름의 함수로 구현해봅시다.

In [20]:
def modified_precision(candidate, reference_list, n):
    clip = count_clip(candidate, reference_list, n)
    total_clip = sum(clip.values())
    
    ct = simple_count(candidate,n)
    total_ct = sum(ct.values())
    
    if total_ct ==0:
        total_ct = 1
        
    return (total_clip/total_ct) # 보정된 정밀도
    # count_clip의 합을 분자로 하고 단순 count의 합을 분모로 하면 보정된 정밀도

In [21]:
result = modified_precision(candidate.split(), list(map(lambda ref: ref.split(), references)),1)
print(result)

0.2857142857142857


소수 값이 나오는데 이는 27의 값을 의미합니다. 이는 앞서 육안으로 계산했던 Example 2에서 Ca의 보정된 정밀도와 동일합니다. 지금까지 보정된 유니그램 정밀도에 대해서 설명하고, 직접 구현까지 해보았습니다.

이제부터 설명에서 언급하는 '정밀도'는 기본적으로 보정된 정밀도(Modified Precision)라고 가정합니다. 정밀도를 보정하므로서 Ca에서 발생하는 단어 중복에 대한 문제점은 해결되었습니다. 하지만 유니그램 정밀도가 가지는 본질적인 문제점있기에 이제는 유니그램을 넘어 바이그램, 트라이그램 등과 같이 n-gram으로 확장해야 합니다. 문제점이 무엇인지 이해하고, 어떻게 n-gram으로 확장하는지 학습해봅시다.

##### 4) 순서를 고려하기 위해서 n-gram으로 확장하기
BoW 표현과 유사하게, 유니그램 정밀도와 같이 각 단어의 빈도수로 접근하는 방법은 결국 단어의 순서를 고려하지 않는다는 특징이 있습니다. Example 1에 Ca3이라는 새로운 문장을 추가해보고 기존의 Ca1과 비교해봅시다.

##### 5) 짧은 문장 길이에 대한 패널티(Brevity Penalty)
n-gram으로 단어의 순서를 고려한다고 하더라도 여전히 남아있는 문제가 있는데, 바로 Ca의 길이에 BLEU의 점수가 과한 영향을 받을 수 있다는 점입니다. 기존 Example 1에 다음의 Ca를 추가한다고 해보겠습니다.

In [33]:
def closest_ref_length(candidate, reference_list):
    ca_len = len(candidate)
    ref_lens = (len(ref) for ref in reference_list)
    
    closest_ref_len = min(ref_lens, key=lambda ref_len:(abs(ref_len - ca_len), ref_len))
    
    return closest_ref_len

만약 Ca와 길이가 정확히 동일한 Ref가 있다면 길이 차이가 0인 최고 수준의 매치(best match length)입니다. 또한 만약 서로 다른 길이의 Ref이지만 Ca와 길이 차이가 동일한 경우에는 더 작은 길이의 Ref를 택합니다. 예를 들어 Ca가 길이가 10인데, Ref 1, 2가 각각 9와 11이라면 길이 차이는 동일하게 1밖에 나지 않지만 9를 택합니다. closest_ref_length 함수를 통해 r을 구했다면, 이제 BP를 구하는 함수 brevity_penalty를 구현해봅시다.



In [34]:
def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    
    ref_len = closest_ref_length(candidate, reference_list)
    
    if ca_len > ref_len:
        return 1
    elif ca_len ==0:
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

위 함수는 앞서 배운 BP의 수식처럼 c가 r보다 클 경우에는 1을 리턴하고, 그 외의 경우에는 $e^{1−r/c}$를 리턴합니다. 이제 최종적으로 BLEU 점수를 계산하는 함수 bleu_score를 구현해봅시다

In [35]:
def bleu_score(candidate, reference_list, weights=[0.25,0.25,0.25,0.25]):
    bp = brevity_penalty(candidate, reference_list)
    
    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights, start=1)]
    
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights,p_n)])
    
    return bp * np.exp(score)

위의 bleu_score 함수는 기본적으로는 N이 4에 각 gram에 대한 가중치는 동일하게 0.25라 주어진다고 가정합니다. 또한 함수 내에서는 BP를 구하고 bp에, p1,p2,...,pn를 구하여 p_n에 저장하도록 구현되어져 있습니다. 그리고 앞서 배운 BLEU의 식에 따라 추가 연산하여 최종 계산한 값을 리턴합니다.

위 함수가 동작하기 위해서는 앞서 구현한 simple_count, count_clip, modified_precision, brevity_penalty 4개의 함수 또한 모두 구현되어져 있어야 합니다. 지금까지 구현한 BLEU 코드로 계산된 점수와 NLTK 패키지에 이미 구현되어져 있는 BLEU 코드로 계산된 점수를 비교해봅시다.

### 2. NLTK를 사용한 BLEU 측정하기
파이썬에서는 NLTK 패키지를 사용하여 BLEU를 계산할 수 있습니다.

In [36]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print(bleu_score(candidate.split(), list(map(lambda ref: ref.split(), references))))
print(bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)), candidate.split()))

0.5045666840058485
0.5045666840058485


실제 BLEU 구현은 이것보다 좀 더 복잡합니다. NLTK 패키지에서의 상세한 BLEU 코드는 아래의 링크에서 확인할 수 있으며 이번 챕터에서 구현한 코드와 실제 구현에서 고려해야할 것들은 무엇이 있는지 비교해봅시다.